## Imports

In [1]:
# Python standard library
import csv
import json
import datetime
import sys

# Scipy
import numpy as np
import sklearn.linear_model
import sklearn.model_selection
import sklearn.utils
import pandas as pd

import matplotlib.pyplot as plt

# Jupyter
import IPython.display

# Program-specific
sys.path.append('../modules')
import constants
import functions

## Load data

In [2]:
df_train = functions.LoadTrainData(aug = True, mix = True)

/home/prestonh/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


## Data frame preprocessing

In [3]:
# Fill NA values w/ file values
fill_file_path = '../data/train/missing_inc_angles.csv'
functions.FillMissing(df_train, fill_file_path)

## K-Folds cross-validation for C

In [4]:
def LogLoss(predicted_probas, labels):
    return -(np.dot(labels, np.log(predicted_probas)) + np.dot((1-labels), np.log(1-predicted_probas)))

In [5]:
Cs = [1e-5*(1.5**i) for i in range(7)]
print(Cs)

[1e-05, 1.5000000000000002e-05, 2.25e-05, 3.375e-05, 5.0625000000000004e-05, 7.593750000000001e-05, 0.00011390625000000001]


In [6]:
kfolds = sklearn.model_selection.KFold(n_splits = 10)

In [7]:
errors = [0 for i in range(len(Cs))]
for i, C in enumerate(Cs):
    
    
    '''
    Benchmarking
    '''
    IPython.display.clear_output()
    print('i = ', str(i), '/', str(len(Cs)), '\t\tC = ', C)
    
    
    
    
    
    for train_indices, valid_indices in kfolds.split(df_train):
    
        
        

        

    

        # Create model
        model = sklearn.linear_model.LogisticRegression(C = C)

        # Fit model
        model.fit(df_train[constants.inputs].iloc[train_indices], df_train[constants.output].iloc[train_indices])

        # Test model
        predictions = model.predict_proba(df_train[constants.inputs].iloc[valid_indices])[:,1]

        # Get errors
        errors[i] += functions.LogLoss(predictions, df_train[constants.output].iloc[valid_indices])
        
        print(errors[i])
        
        break
        
        
    break
        

i =  0 / 7 		C =  1e-05
0.558663622868


In [ ]:
plt.semilogx(Cs, errors, marker = 'o')
plt.xlabel('C')
plt.ylabel('Log loss')
plt.grid()
plt.show()

In [ ]:
for i in range(len(Cs)):
    print(Cs[i], '\t\t', errors[i])

## Make predictions

In [ ]:
df_train = functions.LoadTrainData(aug = False, mix = True)
df_test = functions.LoadTestData(mix = False)

In [ ]:
inputs = constants.inputs
output = constants.output

In [ ]:
# Fill NA values w/ file values
fill_file_path = '../data/missing_inc_angles.csv'
functions.FillMissing(df_train, fill_file_path)

In [ ]:
# Create
model = sklearn.linear_model.LogisticRegression(C = 5e-5)

# Fit
model.fit(df_train[inputs], df_train[output])

# Test
predictions = model.predict_proba(df_test[inputs])[:,1]

## Write to file

In [ ]:
output_file_path = '../data/submissions/submission_' + str(datetime.datetime.now().date()) + '_' + str(datetime.datetime.now().time()).replace(':', '-').split('.')[0]

with open(output_file_path, 'w') as output_file_handle:
    file_writer = csv.writer(output_file_handle, delimiter = ',')
    
    file_writer.writerow(['id', 'is_iceberg'])
    
    for i in range(len(predictions)):
        file_writer.writerow([df_test.index.values[i], predictions[i]])
        